In [46]:
import mysql.connector, hashlib, math, time

class Database:
    def __init__(self):
        self.user = False
        
        host="localhost"
        user="root"
        password=""
        database="sklep"
        self.con = mysql.connector.connect(host=host, user=user, password=password, database=database)
        self.cursor = self.con.cursor()

    def query(self, q):
        self.cursor.execute(q)
        return self.cursor.fetchall()

    def close(self):
        self.cursor.close()
        self.con.close()

    def login(self, l, p):
        result = self.query(f"select id, nazwa "+
                            f"from users "+
                            f"where (login='{l}' or email='{l}') and haslo=md5('{p}')")
        if len(result) > 1:
            print("Database error!")
        if len(result) == 1:
            self.user = result[0]
        self.getUprawnienia()
        return self.user

    def register(self, l, p, n, t):
        try:
            self.query(f"insert into users (login, haslo, nazwa, typ) "+
                       f"values ('{l}', md5('{p}'), '{n}', '{t}')")
            self.con.commit()
            return self.login(l, p)
        except:
            return False

    def getUprawnienia(self):
        if self.user == False:
            return False

        result = self.query(f"select typyk.users, typyk.zamowienia "+
                            f"from users inner join typyk on users.typ = typyk.id "+
                            f"where users.id = '{self.user[0]}'")

        self.uprawnienia = []
        for r in result[0]:
            if r == "T":
                self.uprawnienia.append(True)
            else:
                self.uprawnienia.append(False)

        return self.uprawnienia

    def getZamowienia(self):
        if self.user == False:
            return False
        query = f"select * from zamowienia where status <> 'Zakonczono'"
        if not self.uprawnienia[1]:
            query += f" and userid = {self.user[0]}"
            
        result = self.query(query)

        return result


database = Database()

print("Logowanie")
while True:
    login = input("Podaj login:")
    if login == "":
        break
    password = input("Podaj haslo:")

    result = database.login(login, password)
    
    if result != False:
        print("Witaj,", result[1])
        break

    print("Nieprawidlowe dane logowania")

if database.user != False:
    for line in database.getZamowienia():
        print(line)
else:
    print("Rejestracja")

    while True:
        login = input("Podaj login:")
        if login == "":
            break
        password = input("Podaj haslo:")
        nazwa = input("Podaj nazwe:")
        typ = input("Podaj typ konta: 1 dla admin, 2 dla user:")

        result = database.register(login, password, nazwa, typ)

        if result != False:
            print("Witaj,", result[1])
            break

        print("Taki login juz istnieje")

database.close()

Logowanie


Podaj login: 


Rejestracja


Podaj login: 
